In [4]:
import numpy as np
import os
CurrentPath = os.getcwd()
data_1 = np.loadtxt(CurrentPath + '/2025/data_1.txt', delimiter=' ')
data_2 = np.loadtxt(CurrentPath + '/2025/data_2.txt', delimiter=' ')
data_1_X = data_1.T[0];data_1_Y = data_1.T[1]; data_1_var = data_1.T[2]
data_2_X = data_2.T[0];data_2_Y = data_2.T[1]; data_2_var = data_2.T[2]

In [32]:
paramA = np.arange(-20,20,1)
paramB = np.arange(-20,20,1)
paramC = np.arange(-20,20,1)

gridA, gridB, gridC = np.meshgrid(paramA, paramB, paramC)
def chisqQuad(gridA, gridB, gridC, X, Y, var): # chisquare for quadratic function
    chisq = 0
    for i, Xval in enumerate(X):
        Ytheory = gridA*Xval**2 + gridB*Xval + gridC
        chisq = chisq + (Y[i] - Ytheory)**2/var[i]
    return chisq

In [33]:
arr = chisqQuad(gridA, gridB, gridC, data_2_X, data_2_Y, data_2_var)

In [34]:
arr_marginAB = 

array([[[-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        ...,
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19]],

       [[-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        ...,
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19]],

       [[-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        ...,
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19]],

       ...,

       [[-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19],
        [-20, -19, -18, ...,  17,  18,  19

5.273169086060416

In [24]:
gridB

array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]],

       [[3, 3, 3],
        [3, 3, 3],
        [3, 3, 3]]])

In [25]:
gridC

array([[[4.  , 4.67, 6.  ],
        [4.  , 4.67, 6.  ],
        [4.  , 4.67, 6.  ]],

       [[4.  , 4.67, 6.  ],
        [4.  , 4.67, 6.  ],
        [4.  , 4.67, 6.  ]],

       [[4.  , 4.67, 6.  ],
        [4.  , 4.67, 6.  ],
        [4.  , 4.67, 6.  ]]])